In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import loguniform
from pandas import read_csv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline
from os import listdir
from sklearn.metrics import roc_curve, auc, log_loss, precision_score,recall_score,f1_score
from sklearn.model_selection import RandomizedSearchCV
import xgboost

import cv2
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


# Reading the data

In [3]:
df = pd.read_excel("X:\Eye_tracking_Processed_Raw_videos\\thesis_final_2.xlsx")

In [4]:
df = df[['Age', 'Sex', 'before', 'after', 'difference', 'memory', 'eeq']]

In [5]:
df['target']=list(map(lambda x: "1" if x>1.2 else "0",df['eeq']))

In [6]:
df['sex'] = list(map(lambda x: 1 if x=="Female" else 0,df['Sex']))

In [7]:
df.drop(['Sex','eeq'],axis=1,inplace=True)

In [8]:
df.columns

Index(['Age', 'before', 'after', 'difference', 'memory', 'target', 'sex'], dtype='object')

In [9]:
y = df['target']
X = df.drop(['target'],axis=1)

# Applying test train split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=28)

In [26]:

def metricss(actual,pred):
    precision=precision_score(actual,pred, average="binary",pos_label='1')
    recall=recall_score(actual,pred, pos_label='1')
    f1=f1_score(actual,pred, average="binary", pos_label='1')
    
    return(precision,recall,f1)

# RF

In [27]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

# Hyperparameter tuning

In [28]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = rfc()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

# Applying model on test data

In [29]:
rf_cv = rf_random.best_estimator_
y_pred = rf_cv.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print("confusion matrix\n",conf_mat)
precision, recall, f1 = metricss(y_test, y_pred)
print("f1 - ",f1)
print("recall - ",recall)
print("precision - ",precision)

confusion matrix
 [[6 0]
 [2 1]]
f1 -  0.5
recall -  0.3333333333333333
precision -  1.0


# Gradient Boosting

In [30]:
parameters = {
    "n_estimators":[5,50,250,500,1000],
    "max_depth":[1,3,5,7,9,15,20,50],
    "learning_rate":[0.01,0.1,1,10,100],
    "max_depth" : [int(x) for x in np.linspace(10, 110, num = 11)],
    "min_samples_split" : [2, 5, 10],
    "min_samples_leaf" : [1, 2, 4]
}

# Hyperparameter tuning

In [31]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
gbc = GradientBoostingClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gbc_random = RandomizedSearchCV(estimator = gbc, param_distributions = parameters, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
gbc_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=GradientBoostingClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.1, 1, 10,
                                                          100],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [5, 50, 250, 500,
                                                         1000]},
                   random_state=42, verbose=1)

# Applying model on test data

In [32]:
gbc_cv = gbc_random.best_estimator_
y_pred = gbc_cv.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print("confusion matrix\n",conf_mat)
precision, recall, f1 = metricss(y_test, y_pred)
print("f1 - ",f1)
print("recall - ",recall)
print("precision - ",precision)

confusion matrix
 [[4 2]
 [3 0]]
f1 -  0.0
recall -  0.0
precision -  0.0


# LR

In [36]:
# random search logistic regression model on the sonar dataset


lr = LogisticRegression()
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = loguniform(1e-5, 100)
# define search
lr_random = RandomizedSearchCV(lr, space, n_iter=100, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)


# Hyperparameter tuning

In [37]:
# execute search
result = lr_random.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.6694444444444444
Best Hyperparameters: {'C': 0.291645572949072, 'penalty': 'l2', 'solver': 'saga'}


C:\Users\aksha\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1410 fits failed out of a total of 3000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
210 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\aksha\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\aksha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\aksha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(

# Applying model on test data

In [38]:
lr_cv = lr_random.best_estimator_
y_pred = lr_cv.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print("confusion matrix\n",conf_mat)
precision, recall, f1 = metricss(y_test, y_pred)
print("f1 - ",f1)
print("recall - ",recall)
print("precision - ",precision)

confusion matrix
 [[6 0]
 [2 1]]
f1 -  0.5
recall -  0.3333333333333333
precision -  1.0
